In [5]:
import pandas as pd
import numpy as np
from itertools import chain, combinations

In [6]:
class Node:
  def __init__(self, col, dtype, values = None):
    self.col = col
    self.dtype = dtype
    if self.dtype == 'categorical':
      assert values is not None, 'Mention values for categorical feature.'
      self.values = values
    else: self.values = None
    self.yes = True
    self.no = False
  
  def __str__(self):
    return f'COLUMN: {self.col}, VALUES: {self.values},'

In [7]:
class DecisionTree:
  def __init__(self):
    self.tree = None

  def __gini(self, cnt):
    total = np.sum(cnt)
    if total == 0: return 0
    return 1 - (cnt[0] / total) ** 2 - (cnt[1] / total) **2

  def __powerset(self, iterable):
    s = list(iterable)
    if len(s) == 1: return [tuple(s)]
    return list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))[1:-1]

  def __total_imp(self, true_count, false_count):
    true_total = np.sum(true_count)
    false_total = np.sum(false_count)
    total = true_total + false_total
    return (self.__gini(true_count) * true_total / total + self.__gini(false_count) * false_total / total,
            self.__gini(true_count), self.__gini(false_count))

  def __get_imp(self, feature, val, data, label_name):
    if self.col_type[feature] == 'numerical':
      pass
    else:
      true_count = [0, 0]
      false_count = [0, 0]
      for i in range(len(data[feature])):
        if data[feature].iloc[i] in val:
          if data[label_name].iloc[i]: true_count[1] += 1
          else: true_count[0] += 1
        else:
          if data[label_name].iloc[i]: false_count[1] += 1
          else: false_count[0] += 1
      return self.__total_imp(true_count, false_count)
  
  def __feature_impurity(self, feature, data, label_name):
    if self.col_type[feature] == 'numerical':
      pass
    else:
      values = self.__powerset(data[feature].unique())
      val_imp = set()
      for val in values:
        imp = self.__get_imp(feature, val, data, label_name)
        val_imp.add((imp, val))
        # print(f'Feature: {feature}, Values: {val}, Impurity: {imp[0]}')
      return val_imp.pop()

  def __build_tree(self, data, label_name, cols, par_imp = 10):
    if len(cols) == 1: return None
    col_imp = set()
    for col in cols:
      if self.col_type[col] == 'label': continue
      col_imp.add((self.__feature_impurity(col, data, label_name), col))
    best = col_imp.pop()
    col = best[1]
    if best[0][0][0] < par_imp:
      node = Node(col, self.col_type[col], best[0][1])
      data_yes = data[data[col].isin(list(best[0][1])) == True].drop(col, axis=1)
      data_no = data[data[col].isin(list(best[0][1])) == False].drop(col, axis=1)
      new_cols = list(data_yes.columns)
      node.yes = self.__build_tree(data_yes, label_name, new_cols.copy(), best[0][0][1])
      node.no = self.__build_tree(data_no, label_name, new_cols.copy(), best[0][0][2])
      if node.yes is None: node.yes = True
      if node.no is None: node.no = False
      return node

  def fit(self, df, label_name):
    self.col_type = {}
    self.cols = list(df.columns)
    for col in self.cols:
      if col == label_name: self.col_type[col] = 'label'
      elif type(df[col][0]) == str:
        self.col_type[col] = 'categorical'
      else: self.col_type[col] = 'numerical'
    self.tree = self.__build_tree(df, label_name, self.cols.copy())

  def __predict(self, data, node):
    if type(node) == bool: return node
    
    val = data[node.col]
    if val in node.values: node = node.yes
    else: node = node.no
    
    return self.__predict(data, node)

  def predict(self, df):
    preds = []
    for i in range(len(df)):
      preds.append(self.__predict(df.iloc[i], self.tree))
    return preds

In [8]:
df = pd.read_csv('data.csv')
df.head()

,Age,Income,Gender,MaritialStatus,Buys
0,<21,High,Male,Single,No
1,<21,High,Male,Married,No
2,21-35,High,Male,Single,Yes
3,>35,Medium,Male,Single,Yes
4,>35,Low,Female,Single,Yes


In [9]:
train_df = df[:-1].copy()
train_df['Buys'] = train_df['Buys'] == 'Yes'

In [10]:
test_df = df[-1:].copy().drop('Buys', axis = 1)

In [11]:
clf = DecisionTree()

In [12]:
clf.fit(train_df, 'Buys')

In [13]:
print(f'Root: {clf.tree.col}')

Root: Income


In [14]:
clf.predict(test_df)

[True]